In [1]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(H5weaver)
quiet_library(ArchR)
quiet_library(purrr)
quiet_library(ATAComb)
quiet_library(SingleCellExperiment)
quiet_library(S4Vectors)

### Initialize ArchRProject

In [3]:
addArchRGenome("hg38")
addArchRThreads(8)

Setting default genome to Hg38.

Setting default number of Parallel threads to 8.



In [4]:
arrows <- list.files("../data/", pattern = "arrow$", full.names = TRUE)
proj <- ArchRProject(arrows)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...



Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 
2 
3 
4 
5 
6 


Getting Cell Metadata...



Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
   

In [5]:
proj <- addIterativeLSI(proj)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-3437749a75e-Date-2021-11-21_Time-21-28-06.log
If there is an issue, please report to github with logFile!

2021-11-21 21:28:06 : Computing Total Across All Features, 0.003 mins elapsed.

2021-11-21 21:28:12 : Computing Top Features, 0.103 mins elapsed.

###########
2021-11-21 21:28:14 : Running LSI (1 of 2) on Top Features, 0.124 mins elapsed.
###########

2021-11-21 21:28:14 : Sampling Cells (N = 10004) for Estimated LSI, 0.126 mins elapsed.

2021-11-21 21:28:14 : Creating Sampled Partial Matrix, 0.126 mins elapsed.

2021-11-21 21:28:27 : Computing Estimated LSI (projectAll = FALSE), 0.349 mins elapsed.

2021-11-21 21:29:33 : Identifying Clusters, 1.45 mins elapsed.

Warning message:
“The following arguments are not used: row.names”
2021-11-21 21:30:00 : Identified 6 Clusters, 1.892 mins elapsed.

2021-11-21 21:30:00 : Saving LSI Iteration, 1.892 mins elapsed.

2021-11-21 21:30:19 : Creating Cluster Matrix on the t

In [6]:
proj


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /home/jupyter/ArchR_2020/Code/ArchROutput 
samples(6): X070-P1_PB1051W10 X070-P1_PB1178W10 ... X070-P1_PB5206W10
  X070-P1_PB7626W10
sampleColData names(1): ArrowFiles
cellColData names(35): Sample well_id ... altius_frac altius_count
numberOfCells(1): 41440
medianTSS(1): 16.662
medianFrags(1): 5543

### Load and add scRNA-seq as GeneExpressionMatrix

In [2]:
h5_files <- list.files("../data/", pattern = ".h5", full.names = TRUE)

In [7]:
proj_meta <- getCellColData(proj)
proj_meta$archr_names <- rownames(proj_meta)

In [8]:
sce <- import10xFeatureMatrix(h5_files)

if(class(sce) == "list") {
    for(i in 2:length(sce)) {
        rowRanges(sce[[i]]) <- rowRanges(sce[[1]])
    }
    sce <- do.call(cbind, sce)
}

Importing Feature Matrix 1 of 6

Importing Feature Matrix 2 of 6

Importing Feature Matrix 3 of 6

Importing Feature Matrix 4 of 6

Importing Feature Matrix 5 of 6

Importing Feature Matrix 6 of 6

Error in combining individual feature matrices! Returning as a list of individual feature matrices!



In [9]:
colnames(sce) <- sub("#","",colnames(sce))
sce <- sce[,proj_meta$barcodes]
colnames(sce) <- proj_meta$archr_names

sce

class: RangedSummarizedExperiment 
dim: 36578 41440 
metadata(0):
assays(1): counts
rownames(36578): MIR1302-2HG FAM138A ... AC007325.4 AC007325.2
rowData names(5): feature_type genome id interval name
colnames(41440): X070-P1_PB1051W10#9f02f32ebde111eba4dc42010a19c80f
  X070-P1_PB1051W10#a1e3aaa2bde111eba4dc42010a19c80f ...
  X070-P1_PB7626W10#90908ca2bde111ebb23e42010a19c80f
  X070-P1_PB7626W10#a3626878bde111eba4dc42010a19c80f
colData names(0):

In [10]:
proj <- addGeneExpressionMatrix(
    proj,
    seRNA = sce
)

ArchR logging to : ArchRLogs/ArchR-addGeneExpressionMatrix-34376a6ec889-Date-2021-11-21_Time-21-32-44.log
If there is an issue, please report to github with logFile!

Overlap w/ scATAC = 1

2021-11-21 21:32:45 : 

Overlap Per Sample w/ scATAC : X070-P1_PB1051W10=6822,X070-P1_PB1178W10=8032,X070-P1_PB1194W10=6355,X070-P1_PB2216W10=6564,X070-P1_PB5206W10=7194,X070-P1_PB7626W10=6473

2021-11-21 21:32:45 : 

2021-11-21 21:32:53 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addGeneExpressionMatrix-34376a6ec889-Date-2021-11-21_Time-21-32-44.log



### Load models

In [18]:
model_files <- list.files("Models", full.names = TRUE)
model_list <- map(model_files, readRDS)
model_names <- map_chr(model_list, "modelName")
names(model_list) <- model_names

### Perform correlations

Save outputs to .csv

In [12]:
skip_models <- ""

In [25]:
out_dir <- "model_cors"
if(!dir.exists(out_dir)) {
    dir.create(out_dir)
}
agsc_formals <- names(formals(addGeneScoreMatrix))

In [27]:
for(gene_model in model_list) {
    model_name <- gene_model$modelName
    
    out_file <- file.path(out_dir, paste0(gene_model$modelName, "_cor.csv"))
    
    if(!file.exists(out_file) & !model_name %in% skip_models) {
        message(model_name)
                
        model_params <- c(list(input = proj), gene_model)
        model_params$force <- TRUE
        
        if(gene_model$FN == "addGeneScoreMatrix") {

            model_params <- model_params[names(model_params) %in% agsc_formals]

            message("adding GeneScores")
            proj2 <- do.call(addGeneScoreMatrix, model_params)

            cor_mat_name <- "GeneScoreMatrix"
        } else {
            
            message("adding FeatureMatrix")
            proj2 <- addFeatureMatrix(
                proj,
                features = gene_model$features
            )
            
            cor_mat_name = "FeatureMatrix"
        }
        
        message("running correlations")

        res <- correlateMatrices(
            proj2,
            useMatrix1 = cor_mat_name,
            useMatrix2 = "GeneExpressionMatrix",
            k = 50,
            verbose = FALSE
        )

        message("writing results")
        
        res <- as.data.frame(res)
        write.csv(res, out_file, row.names = FALSE, quote = FALSE)
    }
}

GeneBody_0_0

ArchR logging to : ArchRLogs/ArchR-addFeatureMatrix-343737e8c1b5-Date-2021-11-22_Time-06-13-48.log
If there is an issue, please report to github with logFile!

2021-11-22 06:13:51 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addFeatureMatrix-343737e8c1b5-Date-2021-11-22_Time-06-13-48.log

running correlations

ArchR logging to : ArchRLogs/ArchR-correlateMatrices-343764a52558-Date-2021-11-22_Time-06-15-39.log
If there is an issue, please report to github with logFile!

Getting Correlations...

Computing Correlation (250 of 25570)

Computing Correlation (500 of 25570)

Computing Correlation (750 of 25570)

Computing Correlation (1000 of 25570)

Computing Correlation (1250 of 25570)

Computing Correlation (1500 of 25570)

Computing Correlation (1750 of 25570)

Computing Correlation (2000 of 25570)

Computing Correlation (2250 of 25570)

Computing Correlation (2500 of 25570)

Computing Correlation (2750 of 25570)

Computing 

In [28]:
sessionInfo()

R version 4.0.5 (2021-03-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
 [1] grid      parallel  stats4    stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] gridExtra_2.3                     uwot_0.1.10                      
 [3] nabor_0.5.0                       SeuratObject_4.0.2               
 [5] Seurat_4.0.4                      BSgenome.Hsapiens.UCSC.hg38_1.4.3
 [7] BSgenome_1.58.0                   rtracklayer_1.50.0